# 載入所需套件

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re 
from urllib.request import urlretrieve
import os

# 定義爬取Beauty板文章資訊函數

In [2]:
def ptt_beauty_crawler(url):   
    form_data={'from':'/bbs/Beauty/index.html','yes':'yes'}    
    session=requests.session()
    ##餵入form_data
    response=session.post('https://www.ptt.cc/ask/over18',form_data)
    response=session.get(url)
    soup=BeautifulSoup(response.text,'html.parser')
    
    entrys=soup.select('div.r-ent')
    
    titles,dates,authors,recommends,links=[],[],[],[],[]

    for entry in entrys:
        try:
            titles.append(entry.select('div.title a')[0].text)
        except:
            titles.append(np.nan)
        try:
            dates.append(entry.select('div.date')[0].text)
        except:
            dates.append(np.nan)
        try:
            authors.append(entry.select('div.author')[0].text)    
        except:
            authors.append(np.nan)    
        try:
            recommends.append(entry.select('div.nrec')[0].text)
        except:
            recommends.append(np.nan)
        try:
            links.append('https://www.ptt.cc'+entry.select('div.title a')[0]['href'])
        except:
            links.append(np.nan)

    df=pd.DataFrame({
        '標題':titles,
        '日期':dates,
        '作者':authors,
        '推數':recommends,
        '網址':links
    })
    
    return df

# 定義爬取多頁Beauty板文章資訊和推數篩選函數

In [3]:
def page_condition_function(page,condition):    
    url='https://www.ptt.cc/bbs/Beauty/index.html'    
    form_data={'from':'/bbs/Beauty/index.html','yes':'yes'}
    session=requests.session()
    #餵入form_data
    response=session.post('https://www.ptt.cc/ask/over18',form_data)

    #df:沒經過資料處理和推數篩選前
    dfs=[]
   
    for i in range(page):    
        d=ptt_beauty_crawler(url)
        dfs.append(d)
        response=session.get(url)
        soup=BeautifulSoup(response.text,'html.parser')
        paging=soup.select('div.btn-group-paging a')
        url='https://www.ptt.cc'+paging[1]['href']
    
    df=pd.concat(dfs,ignore_index=True)
    
    #df_c:經過資料處理和推數篩選後
    #將有NaN的文章資料從表格移除
    df_c=df.dropna(axis=0)
    #將推數為空字串的文章資料從表格移除
    df_c=df_c[df_c['推數']!='']
    #將推數為爆的改成99
    df_c['推數'].replace('爆','99',inplace=True)
    #將推數為X的文章資料從表格移除
    df_c=df_c[~df_c['推數'].str.startswith('X')]
    #將表格篩選只留下推數>=condition的文章資料
    df_c=df_c[(pd.to_numeric(df_c['推數'],errors='coerce')>=condition)]
    #重新設定新表格索引順序
    df_c.reset_index(drop=True,inplace=True)
    
    return df,df_c   

# 指定推數篩選條件下總頁數表格呈現

In [7]:
df,df_c=page_condition_function(5,50)

In [8]:
df.head()

,標題,日期,作者,推數,網址
0,[正妹] Lia 崔智秀,8/12,kaeun421,19,https://www.ptt.cc/bbs/Beauty/M.1565545398.A.D...
1,[正妹] 阿部瑪利亞,8/12,Steven1368,36,https://www.ptt.cc/bbs/Beauty/M.1565546636.A.1...
2,[神人] 求神兄弟象球迷_清秀小姐姐,8/12,dddd506,6,https://www.ptt.cc/bbs/Beauty/M.1565548726.A.D...
3,[廣告] 性感指數破表！平嶋夏海超辣身材藏不住,8/12,kelseyaya,8,https://www.ptt.cc/bbs/Beauty/M.1565573669.A.B...
4,[公告] 開放噓文暫停X1條款,5/13,hateOnas,XX,https://www.ptt.cc/bbs/Beauty/M.1557742996.A.6...


In [9]:
df_c.head()

,標題,日期,作者,推數,網址
0,[正妹] 肉感,8/11,mistsea,64,https://www.ptt.cc/bbs/Beauty/M.1565518826.A.7...
1,[帥哥] 世界上沒有基黑 基努李維,8/11,jack5u06d93,76,https://www.ptt.cc/bbs/Beauty/M.1565537040.A.E...
2,[正妹] 求圖 比五的手勢,8/11,apro0401,99,https://www.ptt.cc/bbs/Beauty/M.1565481224.A.3...
3,[正妹] IG滑到......,8/09,a2914436,59,https://www.ptt.cc/bbs/Beauty/M.1565365311.A.3...
4,[正妹] 乃木坂46 齋藤飛鳥 阿羞羞,8/10,RedJackson,99,https://www.ptt.cc/bbs/Beauty/M.1565366810.A.5...


# 爬取Beauty版文章圖片(在指定推數以上)

In [88]:
titles=df_c['標題'].values.tolist()
links=df_c['網址'].values.tolist()


form_data={'from':'/bbs/Beauty/index.html','yes':'yes'}
reg_imgur_file=re.compile('http[s]?://[i.]*imgur.com/\w+\.(?:jpg|png|gif)')
session=requests.session()
#餵入form_data
response=session.post('https://www.ptt.cc/ask/over18',form_data)


for link,title in zip(links[:3],titles[:3]):
    response=session.get(link)
    images=reg_imgur_file.findall(response.text)
    print(title)
    directory='Beauty版文章圖片/{}'.format(title)
    
    if not os.path.isdir(directory):
        os.makedirs(directory)
    
    for image in set(images):
        ID=re.search('http[s]?://[i.]*imgur.com/(\w+\.(?:jpg|png|gif))',image).group(1)
        print(ID)
        #連結圖片檔案路徑
        urlretrieve(image,os.path.join(directory,ID))
    
    print('\n')

[正妹] 鍾瑶
YgamB71.jpg
GypSXzI.jpg
ZamLNFp.jpg
XEcs5wx.jpg
u8AmUga.jpg
se2MJfz.jpg
Zprf42T.jpg
R6quOIx.jpg
hoM1o5y.jpg
nGJPZkT.jpg


[神人] 泳裝自拍
ivL3BUS.jpg


[廣告] 性感人妻熊田曜子 身材依舊完美
iuXQDJO.jpg
jkIIm1v.jpg
hDwG36B.jpg
VhwPB2V.jpg
7Z68nNG.jpg
0yCcNsG.jpg
wyiLML4.jpg
wx9BlQQ.jpg
PYWLnpj.jpg
gFE32Gg.jpg
7zfSjSs.jpg
aJ9ht5B.jpg
dWTZA0l.jpg
Bw0LmHc.jpg
SO00IXl.jpg
pNfF3gX.jpg
GboiKQD.jpg
kJFZj1Y.jpg
IlR2LEo.jpg
wIVuItb.jpg
U4BvhKy.jpg
LS1wbnb.jpg


